In [3]:
# Imports
import os 
import torch
from torch import nn
import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy
from pytorch_lightning.callbacks import ModelCheckpoint
import torchvision
from matplotlib import pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
from sym import LCN, ProgressBar, Classification_report, Logger
device = "cuda:0" if torch.cuda.is_available() else "cpu"
plt.style.use("dark_paper")

In [40]:
class LocalLinear(nn.Module):
    def __init__(self,in_features,local_features,kernel_size,padding=0,stride=1,bias=True):
        super(LocalLinear, self).__init__()
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

        fold_num = (in_features+2*padding-self.kernel_size)//self.stride+1
        self.weight = nn.Parameter(torch.randn(fold_num,kernel_size,local_features))
        self.bias = nn.Parameter(torch.randn(fold_num,local_features)) if bias else None
    def forward(self, x:torch.Tensor):
        x = torch.nn.functional.pad(x,[self.padding]*2,value=0)
        x = x.unfold(-1,size=self.kernel_size,step=self.stride)
        x = torch.matmul(x.unsqueeze(2),self.weight).squeeze(2)+self.bias
        return x

/home/kitouni/anaconda3/envs/gnn/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
testset = torch.load("./data/testset.pt")
trainset = torch.load("./data/trainset.pt")

In [6]:
# Aggregate validation losses -> DONE!
# Change back to SGD -> DONE!
# Shit on computaional separation paper
# Reporoduce their experiments
# Show that the bounds are consistent
# By Friday have 5 pages
# Fix LCN Code
# Think about better problem statement

In [17]:
class Model(pl.LightningModule):
    def __init__(self, model, nfilters = 32,learning_rate=1e-2):
        super().__init__()
        self.model = model(nfilters=nfilters)
        self.lr = learning_rate
        self.hparams["Params"] = sum([x.size().numel() for x in self.model.parameters()])
        self.save_hyperparameters({"nfilters" : nfilters, "Model" : self.model.__repr__().replace("\n", "")})
    def forward(self, x):
        return self.model(x)
    def training_step(self, batch, batch_idx):
        x, y = batch
        yhat = self(x)
        loss = self.Loss(yhat, y)
        self.log('train_loss', loss)
        return loss
    def validation_step(self, batch, batch_idx):
        x, y = batch
        yhat = self(x)
        loss = self.Loss(yhat, y)
        preds = torch.argmax(yhat, dim=1)
        acc = accuracy(preds, y)
        # Calling self.log will surface up scalars for you in TensorBoard
        metrics = {'val_loss': loss, 'val_acc': acc}
        self.log_dict(metrics, prog_bar=True,logger=True,on_epoch=True,on_step=False)
        try:
            self.logger.log_hyperparams(self.hparams,metrics=metrics)
        except:
            self.logger.log_hyperparams(self.hparams)
        return metrics
    def validation_epoch_end(self, outputs):
        val_loss_mean = 0
        val_acc_mean = 0
        for output in outputs:
            val_loss_mean += output['val_loss']
            val_acc_mean += output['val_acc']
        val_loss_mean /= len(outputs)
        val_acc_mean /= len(outputs)
        metrics = {'val_loss': val_loss_mean, 'val_acc': val_acc_mean}
#         self.log_dict(metrics, prog_bar=True,logger=True,on_epoch=True,on_step=False)
#         self.logger.log_hyperparams(self.hparams,metrics=metrics)
        return
    def test_step(self, batch, batch_idx):
        # Here we just reuse the validation_step for testing
        return self.validation_step(batch, batch_idx)
    def configure_optimizers(self, learning_rate=1e-1):
        optimizer = torch.optim.SGD(self.model.parameters(), lr=learning_rate)
        return optimizer
    def setup(self, stage=None):
        # Assign train/val datasets for use in dataloaders
        if stage == 'fit' or stage is None:
            self.mnist_train, self.mnist_val = torch.utils.data.random_split(trainset, [
                                                                             50000, 10000])
        # Assign test dataset for use in dataloader(s)
        if stage == 'test' or stage is None:
            self.mnist_test = testset
    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.mnist_train, num_workers=6, batch_size=320)
    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.mnist_val, num_workers=6, batch_size=320,)
    def test_dataloader(self):
        return torch.utils.data.DataLoader(self.mnist_test, num_workers=6, batch_size=320)
    def Loss(self,preds,targets):
        preds = torch.nn.functional.softmax(preds,dim=-1)
        targets = torch.nn.functional.one_hot(targets,10)
        loss = ((preds-targets)**2).mean()
#         loss = torch.nn.CrossEntropyLoss()(preds,targets)
        return loss 

/home/kitouni/anaconda3/envs/gnn/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
class SFCN(torch.nn.Module):
    def __init__(self,input_dim=280,nfilters=32,pool_size=100,C=10):
        super().__init__()
        self.nfilters = nfilters
        self.pool_size = pool_size
        self.fc1 = torch.nn.Linear(input_dim**2, nfilters*pool_size)
        self.pool = torch.nn.MaxPool1d(pool_size,1)
        self.out = torch.nn.Linear(nfilters, C)
    def forward(self,x):
        x = x.flatten(start_dim=1)
        x = torch.nn.functional.relu(self.fc1(x))
        x = self.pool(x.view((-1,self.nfilters,self.pool_size)))
        return self.out(x.view(-1,self.nfilters))

In [9]:
class CNN(torch.nn.Module):
    def __init__(self,nfilters=32):
        super().__init__()
        self.nfilters = nfilters
        self.model = torch.nn.Sequential(torch.nn.Conv2d(1, nfilters, kernel_size=28, stride=28),
                          #torch.nn.BatchNorm2d(nfilters),
                          torch.nn.ReLU(),
                          torch.nn.MaxPool2d(10, 1),
                          torch.nn.Flatten(),
                          torch.nn.Linear(nfilters, 10))
    def forward(self,x):
        return self.model(x)

In [42]:
class PrintLayer(nn.Module):
    def __init__(self):
        super(PrintLayer, self).__init__()
    
    def forward(self, x):
        # Do your print / debug stuff here
        print(x.shape)
        return x

/home/kitouni/anaconda3/envs/gnn/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
logdir = "./lightning_logs/MSE"
if not os.path.exists(logdir): os.mkdir(logdir)
epochs = 10

# LCN2

In [41]:
for nfilters in 2**np.arange(5,0,-1):
    pl.seed_everything(42)
    lcn = Model(LCN2,nfilters)
    trainer = pl.Trainer(auto_lr_find=True)
    trainer.tune(lcn)
    lr_best = lcn.lr
    bar = ProgressBar()
    log = Logger(logdir, name="LCN2",default_hp_metric=False)
    path = logdir+f"/LCN2/version_{log.version}"
    checkpoint_callback = ModelCheckpoint(dirpath=path,filename="weights.cpkt",save_top_k=1,monitor='val_loss',mode='min')
    trainer = pl.Trainer(logger=log, gpus=1, max_epochs=epochs, progress_bar_refresh_rate=1,callbacks=[bar,checkpoint_callback])
    lcn = Model(LCN,nfilters,learning_rate=lr_best)
    trainer.fit(lcn)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
/home/kitouni/anaconda3/envs/gnn/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: GPU available but not used. Set the --gpus flag when calling the script.
  warnings.warn(*args, **kwargs)

  | Name  | Type | Params
-------------------------------
0 | model | LCN2 | 330   


RuntimeError: The size of tensor a (280) must match the size of tensor b (0) at non-singleton dimension 3

# CNN

In [18]:
cnn = Model(CNN,nfilters)
trainer = pl.Trainer(auto_lr_find=True)
trainer.tune(cnn)
lr_best = cnn.lr

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
/home/kitouni/anaconda3/envs/gnn/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: GPU available but not used. Set the --gpus flag when calling the script.
  warnings.warn(*args, **kwargs)

  | Name  | Type | Params
-------------------------------
0 | model | CNN  | 1.6 K 


Learning rate set to 0.030199517204020192


In [22]:
for nfilters in 2**np.arange(5,0,-1):
    pl.seed_everything(10)
    bar = ProgressBar()
    log = Logger(logdir, name="CNN",default_hp_metric=False)
    path = logdir+f"/CNN/version_{log.version}"
    checkpoint_callback = ModelCheckpoint(dirpath=path,filename="weights.cpkt",save_top_k=1,monitor='val_loss',mode='min')
    trainer = pl.Trainer(logger=log, gpus=1, max_epochs=epochs, progress_bar_refresh_rate=1,callbacks=[bar,checkpoint_callback])
    cnn = Model(CNN,nfilters,learning_rate=lr_best)
    trainer.fit(cnn)

/home/kitouni/anaconda3/envs/gnn/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | CNN  | 25.4 K


Epoch 9: 100%|##########| 189/189 [00:07<00:00, 23.89it/s, loss=0.009, v_num=5, val_loss=0.0117, val_acc=0.933]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | CNN  | 12.7 K



Epoch 9: 100%|##########| 189/189 [00:07<00:00, 24.11it/s, loss=0.015, v_num=6, val_loss=0.0194, val_acc=0.886]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | CNN  | 6.4 K 



Epoch 9: 100%|##########| 189/189 [00:07<00:00, 24.35it/s, loss=0.059, v_num=7, val_loss=0.059, val_acc=0.447] 

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | CNN  | 3.2 K 



Epoch 9: 100%|##########| 189/189 [00:07<00:00, 24.02it/s, loss=0.082, v_num=8, val_loss=0.0823, val_acc=0.202]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | CNN  | 1.6 K 



Epoch 9: 100%|##########| 189/189 [00:07<00:00, 23.87it/s, loss=0.091, v_num=9, val_loss=0.0905, val_acc=0.0968]


# LCN 

In [23]:
lcn = Model(LCN,nfilters)
trainer = pl.Trainer(auto_lr_find=True)
trainer.tune(lcn)
lr_best = lcn.lr

/home/kitouni/anaconda3/envs/gnn/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
/home/kitouni/anaconda3/envs/gnn/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: GPU available but not used. Set the --gpus flag when calling the script.
  warnings.warn(*args, **kwargs)

  | Name  | Type | Params
-------------------------------
0 | model | LCN  | 159 K 


Learning rate set to 1.445439770745928e-06


In [24]:
for nfilters in 2**np.arange(5,0,-1):
    pl.seed_everything(42)
    bar = ProgressBar()
    log = Logger(logdir, name="LCN",default_hp_metric=False)
    path = logdir+f"/LNN/version_{log.version}"
    checkpoint_callback = ModelCheckpoint(dirpath=path,filename="weights.cpkt",save_top_k=1,monitor='val_loss',mode='min')
    trainer = pl.Trainer(logger=log, gpus=1, max_epochs=epochs, progress_bar_refresh_rate=1,callbacks=[bar,checkpoint_callback])
    lcn = Model(LCN,nfilters,learning_rate=lr_best)
    trainer.fit(lcn)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | LCN  | 2.5 M 


Epoch 9: 100%|##########| 189/189 [00:14<00:00, 12.76it/s, loss=0.181, v_num=5, val_loss=0.18, val_acc=0.0996]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | LCN  | 1.3 M 



Epoch 9: 100%|##########| 189/189 [00:11<00:00, 16.82it/s, loss=0.173, v_num=6, val_loss=0.173, val_acc=0.134]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

Epoch 9: 100%|##########| 189/189 [00:09<00:00, 19.70it/s, loss=0.170, v_num=7, val_loss=0.172, val_acc=0.137]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | LCN  | 318 K 



Epoch 9: 100%|##########| 189/189 [00:08<00:00, 21.73it/s, loss=0.161, v_num=8, val_loss=0.16, val_acc=0.14]  

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | LCN  | 159 K 



Epoch 9: 100%|##########| 189/189 [00:08<00:00, 22.96it/s, loss=0.142, v_num=9, val_loss=0.142, val_acc=0.123]


# FCN

In [25]:
fcn = Model(SFCN,nfilters)
trainer = pl.Trainer(auto_lr_find=True)
trainer.tune(fcn)
lr_best = fcn.lr

/home/kitouni/anaconda3/envs/gnn/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
/home/kitouni/anaconda3/envs/gnn/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: GPU available but not used. Set the --gpus flag when calling the script.
  warnings.warn(*args, **kwargs)

  | Name  | Type | Params
-------------------------------
0 | model | SFCN | 15.7 M


Learning rate set to 0.19054607179632482


In [27]:
for nfilters in 2**np.arange(5,0,-1):
    pl.seed_everything(42)
    trainer = pl.Trainer(auto_lr_find=True)
    trainer.tune(fcn)
    lr_best = fcn.lr
    bar = ProgressBar()
    log = Logger(logdir, name="FCN",default_hp_metric=False)
    path = logdir+f"/FCN/version_{log.version}"
    checkpoint_callback = ModelCheckpoint(dirpath=path,filename="weights.cpkt",save_top_k=1,monitor='val_loss',mode='min')
    trainer = pl.Trainer(logger=log, gpus=1, max_epochs=epochs, progress_bar_refresh_rate=1,callbacks=[bar,checkpoint_callback])
    fcn = Model(SFCN,nfilters,learning_rate=lr_best)
    trainer.fit(fcn)

/home/kitouni/anaconda3/envs/gnn/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
/home/kitouni/anaconda3/envs/gnn/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: GPU available but not used. Set the --gpus flag when calling the script.
  warnings.warn(*args, **kwargs)

  | Name  | Type | Params
-------------------------------
0 | model | SFCN | 15.7 M


Learning rate set to 1.5848931924611133e-07
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | SFCN | 250 M 


Epoch 1:  90%|######### | 171/189 [00:12<00:01, 13.17it/s, loss=0.128, v_num=10, val_loss=0.141, val_acc=0.27]


Epoch 1:  92%|#########2| 174/189 [00:13<00:01, 13.27it/s, loss=0.128, v_num=10, val_loss=0.141, val_acc=0.27]

Epoch 2:  87%|########7 | 165/189 [00:12<00:01, 12.88it/s, loss=0.118, v_num=10, val_loss=0.132, val_acc=0.31]


Epoch 2:  89%|########8 | 168/189 [00:12<00:01, 12.99it/s, loss=0.118, v_num=10, val_loss=0.132, val_acc=0.31]

Epoch 3:  84%|########4 | 159/189 [00:12<00:02, 12.63it/s, loss=0.105, v_num=10, val_loss=0.123, val_acc=0.34]


Epoch 3:  86%|########5 | 162/189 [00:12<00:02, 12.72it/s, loss=0.105, v_num=10, val_loss=0.123, val_acc=0.34]

Epoch 4:   0%|          | 0/189 [00:00<?, ?it/s, loss=0.105, v_num=10, val_loss=0.117, val_acc=0.383]          



Epoch 4:   1%|          | 1/189 [00:01<03:28,  1.11s/it, loss=0.105, v_num=10, val_loss=0.117, val_acc=0.383]

Validation ...: 0it [00:00, ?it/s][00:11<00:02, 13.66it/s, loss=0.082, v_num=10, val_loss=0.117, va

GPU available: True, used: False
TPU available: False, using: 0 TPU cores

  | Name  | Type | Params
-------------------------------
0 | model | SFCN | 250 M 


Learning rate set to 6.918309709189366e-07
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | SFCN | 125 M 


...

Epoch 0:  84%|########4 | 159/189 [00:08<00:01, 18.76it/s, loss=0.131, v_num=13]



Epoch 1:   0%|          | 0/189 [00:00<?, ?it/s, loss=0.131, v_num=13, val_loss=0.132, val_acc=0.285]          



Epoch 1:   1%|1         | 2/189 [00:01<01:57,  1.60it/s, loss=0.131, v_num=13, val_loss=0.132, val_acc=0.285]

Validation ...: 0it [00:00, ?it/s][00:07<00:01, 21.96it/s, loss=0.118, v_num=13, val_loss=0.132, val_acc=0.285]





Epoch 2:   0%|          | 0/189 [00:00<?, ?it/s, loss=0.118, v_num=13, val_loss=0.124, val_acc=0.32]           





Validation ...: 0it [00:00, ?it/s][00:07<00:01, 22.00it/s, loss=0.110, v_num=13, val_loss=0.124, val_acc=0.32]





Epoch 3:   0%|          | 0/189 [00:00<?, ?it/s, loss=0.110, v_num=13, val_loss=0.106, val_acc=0.379]          





Validation ...: 0it [00:00, ?it/s][00:07<00:01, 21.87it/s, loss=0.101, v_num=13, val_loss=0.106, val_acc=0.379]






Epoch 9: 100%|##########| 189/189 [00:10<00:00, 18.78it/s, loss=0.067, v_num=13, val_loss=0.0686, val_acc

GPU available: True, used: False
TPU available: False, using: 0 TPU cores



  | Name  | Type | Params
-------------------------------
0 | model | SFCN | 31.4 M


Learning rate set to 7.585775750291837e-08
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | SFCN | 15.7 M


Epoch 0:   1%|1         | 2/189 [00:01<02:01,  1.54it/s, loss=0.134, v_num=14]



Epoch 0:   2%|2         | 4/189 [00:01<01:03,  2.90it/s, loss=0.132, v_num=14]
Validation ...: 0it [00:00, ?it/s][00:06<00:01, 23.67it/s, loss=0.103, v_num=14]



Epoch 0:  84%|########4 | 159/189 [00:07<00:01, 20.23it/s, loss=0.103, v_num=14]

Epoch 1:   0%|          | 0/189 [00:00<?, ?it/s, loss=0.103, v_num=14, val_loss=0.11, val_acc=0.2]          





Validation ...: 0it [00:00, ?it/s][00:06<00:01, 23.39it/s, loss=0.093, v_num=14, val_loss=0.11, val_acc=0.2]





Epoch 2:   0%|          | 0/189 [00:00<?, ?it/s, loss=0.093, v_num=14, val_loss=0.0977, val_acc=0.268]          





Validation ...: 0it [00:00, ?it/s][00:06<00:01, 23.67it/s, loss=0.074, v_num=14, val_loss=0.0977, val_acc=0.268]






Epoch 9: 100%|##########| 189/189 [00:09<00:00, 20.41it/s, loss=0.067, v_num=14, val_loss=0.0689, val_acc=0.37] 


# Cross Entropy 

In [8]:
for nfilters in 2**np.arange(5,0,-1):
    pl.seed_everything(42)
    bar = ProgressBar()
    log = Logger(logdir, name="CNN",default_hp_metric=False)
    path = logdir+f"/CNN/version_{log.version}"
    checkpoint_callback = ModelCheckpoint(dirpath=path,filename="weights.cpkt",save_top_k=1,monitor='val_loss',mode='min')
    trainer = pl.Trainer(logger=log, gpus=1, max_epochs=epochs, progress_bar_refresh_rate=1,callbacks=[bar,checkpoint_callback])
    cnn = Model(CNN,nfilters)
    trainer.fit(cnn)

Missing logger folder: ./lightning_logs/CNN
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | CNN              | 25.4 K
1 | Loss  | CrossEntropyLoss | 0     


Epoch 9: 100%|##########| 188/188 [00:08<00:00, 21.82it/s, loss=0.710, v_num=0, val_loss=0.716, val_acc=0.79] 

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | CNN              | 12.7 K
1 | Loss  | CrossEntropyLoss | 0     



Epoch 9: 100%|##########| 188/188 [00:08<00:00, 21.96it/s, loss=1.767, v_num=1, val_loss=1.76, val_acc=0.255]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | CNN              | 6.4 K 
1 | Loss  | CrossEntropyLoss | 0     



Epoch 9: 100%|##########| 188/188 [00:08<00:00, 21.91it/s, loss=2.301, v_num=2, val_loss=2.3, val_acc=0.115]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | CNN              | 3.2 K 
1 | Loss  | CrossEntropyLoss | 0     



Epoch 9: 100%|##########| 188/188 [00:08<00:00, 21.47it/s, loss=2.302, v_num=3, val_loss=2.3, val_acc=0.111]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | CNN              | 1.6 K 
1 | Loss  | CrossEntropyLoss | 0     



Epoch 9: 100%|##########| 188/188 [00:08<00:00, 21.82it/s, loss=2.302, v_num=4, val_loss=2.3, val_acc=0.111]


In [9]:
for nfilters in 2**np.arange(5,0,-1):
    pl.seed_everything(42)
    bar = ProgressBar()
    log = Logger(logdir, name="LCN",default_hp_metric=False)
    path = logdir+f"/LNN/version_{log.version}"
    checkpoint_callback = ModelCheckpoint(dirpath=path,filename="weights.cpkt",save_top_k=1,monitor='val_loss',mode='min')
    trainer = pl.Trainer(logger=log, gpus=1, max_epochs=epochs, progress_bar_refresh_rate=1,callbacks=[bar,checkpoint_callback])
    lcn = Model(LCN,nfilters)
    trainer.fit(lcn)

Missing logger folder: ./lightning_logs/LCN
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | LCN              | 2.5 M 
1 | Loss  | CrossEntropyLoss | 0     


Epoch 9: 100%|##########| 188/188 [00:15<00:00, 12.19it/s, loss=1.546, v_num=0, val_loss=2.05, val_acc=0.432]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | LCN              | 1.3 M 
1 | Loss  | CrossEntropyLoss | 0     



Epoch 9: 100%|##########| 188/188 [00:12<00:00, 15.60it/s, loss=1.831, v_num=1, val_loss=2.25, val_acc=0.319]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | LCN              | 636 K 
1 | Loss  | CrossEntropyLoss | 0     



Epoch 9: 100%|##########| 188/188 [00:10<00:00, 18.52it/s, loss=2.032, v_num=2, val_loss=2.09, val_acc=0.217]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | LCN              | 318 K 
1 | Loss  | CrossEntropyLoss | 0     



Epoch 9: 100%|##########| 188/188 [00:09<00:00, 20.32it/s, loss=2.159, v_num=3, val_loss=2.26, val_acc=0.157]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | LCN              | 159 K 
1 | Loss  | CrossEntropyLoss | 0     



Epoch 9: 100%|##########| 188/188 [00:08<00:00, 21.35it/s, loss=2.213, v_num=4, val_loss=2.29, val_acc=0.136]


In [10]:
for nfilters in 2**np.arange(5,0,-1):
    pl.seed_everything(42)
    bar = ProgressBar()
    log = Logger(logdir, name="FCN",default_hp_metric=False)
    path = logdir+f"/FCN/version_{log.version}"
    checkpoint_callback = ModelCheckpoint(dirpath=path,filename="weights.cpkt",save_top_k=1,monitor='val_loss',mode='min')
    trainer = pl.Trainer(logger=log, gpus=1, max_epochs=epochs, progress_bar_refresh_rate=1,callbacks=[bar,checkpoint_callback])
    fcn = Model(SFCN,nfilters)
    trainer.fit(fcn)

Missing logger folder: ./lightning_logs/FCN
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | SFCN             | 250 M 
1 | Loss  | CrossEntropyLoss | 0     


Epoch 9: 100%|##########| 188/188 [00:18<00:00,  9.91it/s, loss=8.814, v_num=0, val_loss=28.9, val_acc=0.904] 


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | SFCN             | 125 M 
1 | Loss  | CrossEntropyLoss | 0     


Epoch 9: 100%|##########| 188/188 [00:13<00:00, 13.44it/s, loss=1.997, v_num=1, val_loss=5.54, val_acc=0.895] 


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | SFCN             | 62.7 M
1 | Loss  | CrossEntropyLoss | 0     


...

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | SFCN             | 31.4 M
1 | Loss  | CrossEntropyLoss | 0     


Epoch 9: 100%|##########| 188/188 [00:10<00:00, 18.20it/s, loss=0.411, v_num=3, val_loss=0.798, val_acc=0.826]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



  | Name  | Type             | Params
-------------------------------------------
0 | model | SFCN             | 15.7 M
1 | Loss  | CrossEntropyLoss | 0     


Epoch 9: 100%|##########| 188/188 [00:10<00:00, 18.51it/s, loss=1.856, v_num=4, val_loss=1.88, val_acc=0.274]
